In [1]:
from pytrends.request import TrendReq
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [2]:
pytrends = TrendReq(hl='en-US', tz=360)

In [3]:
#mens_clothing_kw_list = ['Nike', 'Adidas', 'Levi\'s', 'Puma', 'Raymond']
#womens_clothing_kw_list = ['Zara', 'H&M', 'Forever 21', 'Marks & Spencer', 'Mango']

In [4]:
state_geo_codes = {
    'Andhra Pradesh': 'IN-AP', 'Arunachal Pradesh': 'IN-AR', 'Assam': 'IN-AS',
    'Bihar': 'IN-BR', 'Chhattisgarh': 'IN-CT', 'Goa': 'IN-GA', 'Gujarat': 'IN-GJ',
    'Haryana': 'IN-HR', 'Himachal Pradesh': 'IN-HP', 'Jharkhand': 'IN-JH',
    'Karnataka': 'IN-KA', 'Kerala': 'IN-KL', 'Madhya Pradesh': 'IN-MP',
    'Maharashtra': 'IN-MH', 'Manipur': 'IN-MN', 'Meghalaya': 'IN-ML',
    'Mizoram': 'IN-MZ', 'Nagaland': 'IN-NL', 'Odisha': 'IN-OR', 'Punjab': 'IN-PB',
    'Rajasthan': 'IN-RJ', 'Sikkim': 'IN-SK', 'Tamil Nadu': 'IN-TN', 'Telangana': 'IN-TG',
    'Tripura': 'IN-TR', 'Uttar Pradesh': 'IN-UP', 'Uttarakhand': 'IN-UT',
    'West Bengal': 'IN-WB'
}

In [5]:
state_name = input("Enter the name of the state (e.g., Maharashtra): ")

In [6]:
geo_code = state_geo_codes.get(state_name)

In [7]:
user_input1 = input("Enter brands available for mens wear (max 5): ")
mens_wear_kw = user_input1.split()


user_input2 = input("Enter brands available for womens wear (max 5): ")
womens_wear_kw = user_input2.split()

In [8]:
if geo_code is None:
    print(f"Invalid state name: {state_name}. Please enter a valid state from the list.")
    
else:
    
    pytrends = TrendReq(hl='en-US', tz=360)

    
    #mens_wear_kw = ['Nike', 'Adidas', 'Puma', 'Reebok', 'Levi\'s']
    #womens_wear_kw = ['Zara', 'H&M', 'Forever 21', 'Mango', 'Fendi']

    
    def analyze_trends(kw_list, category, geo_code):
        
        pytrends.build_payload(kw_list, cat=0, timeframe='today 12-m', geo=geo_code, gprop='')

        
        data = pytrends.interest_over_time()

        
        if 'isPartial' in data.columns:
            data = data.drop(columns=['isPartial'])

        
        df = pd.DataFrame(data)

        
        most_trending_brand = df.sum().idxmax()
        
       
        print(f"The most trending {category} brand in {state_name} is {most_trending_brand}. Buy it earliest!")

        return df

    
    mens_wear_data = analyze_trends(mens_wear_kw, 'Men\'s Wear', geo_code)
    womens_wear_data = analyze_trends(womens_wear_kw, 'Women\'s Wear', geo_code)

    
    def plot_top_2_trends(df, category):
        # Identify the top 2 most searched items
        top_2 = df.sum().nlargest(2).index

        
        fig = go.Figure()

        
        for product in top_2:
            fig.add_trace(go.Scatter(x=df.index, y=df[product], mode='lines', name=product))

        
        fig.update_layout(
            title=f'Search Interest Over Time for Top 2 Searched {category} Items ({state_name})',
            xaxis_title='Date',
            yaxis_title='Search Interest',
            legend_title=f'{category} Item',
            xaxis_tickangle=-45
        )

        
        fig.show()

    # Plot for men's wear
    plot_top_2_trends(mens_wear_data, 'Men\'s Wear')

    # Plot for women's wear
    plot_top_2_trends(womens_wear_data, 'Women\'s Wear')    

The most trending Men's Wear brand in Maharashtra is Nike. Buy it earliest!
The most trending Women's Wear brand in Maharashtra is Zara. Buy it earliest!


In [10]:
def analyze_trends(kw_list, category, geo_code):
        pytrends.build_payload(kw_list, cat=0, timeframe='today 12-m', geo=geo_code, gprop='')
        data = pytrends.interest_over_time()

        if 'isPartial' in data.columns:
            data = data.drop(columns=['isPartial'])

        df = pd.DataFrame(data)
        most_trending_brand = df.sum().idxmax()
        print(f"The most trending {category} brand in {state_name} is {most_trending_brand}. Buy it earliest!")
        return df

In [9]:
from statsmodels.tsa.arima.model import ARIMA
import numpy as np

In [11]:
mens_wear_data = analyze_trends(mens_wear_kw, 'Men\'s Wear', geo_code)
womens_wear_data = analyze_trends(womens_wear_kw, 'Women\'s Wear', geo_code)

The most trending Men's Wear brand in Maharashtra is Nike. Buy it earliest!
The most trending Women's Wear brand in Maharashtra is Zara. Buy it earliest!


In [12]:
def plot_top_2_trends(df, category):
        top_2 = df.sum().nlargest(2).index
        fig = go.Figure()

        for product in top_2:
            fig.add_trace(go.Scatter(x=df.index, y=df[product], mode='lines', name=product))

            # Forecasting with ARIMA
            model = ARIMA(df[product], order=(5, 1, 0))  # Adjust order as needed
            model_fit = model.fit()

            forecast = model_fit.forecast(steps=3)  # Forecast for the next 3 months
            forecast_index = pd.date_range(start=df.index[-1] + pd.Timedelta(days=1), periods=3, freq='M')
            
            fig.add_trace(go.Scatter(x=forecast_index, y=forecast, mode='lines', name=f'Forecast {product}', line=dict(dash='dash')))

        fig.update_layout(
            title=f'Search Interest Over Time for Top 2 Searched {category} Items ({state_name})',
            xaxis_title='Date',
            yaxis_title='Search Interest',
            legend_title=f'{category} Item',
            xaxis_tickangle=-45
        )
        
        fig.show()

In [16]:
# Plot for men's wear
plot_top_2_trends(mens_wear_data, 'Men\'s Wear')

    # Plot for women's wear
plot_top_2_trends(womens_wear_data, 'Women\'s Wear')

c:\Users\vaibh\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency W-SUN will be used.

c:\Users\vaibh\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency W-SUN will be used.

c:\Users\vaibh\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency W-SUN will be used.

c:\Users\vaibh\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency W-SUN will be used.

c:\Users\vaibh\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency W-SUN will be used.

c:\Users\vaibh\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequ

c:\Users\vaibh\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency W-SUN will be used.

c:\Users\vaibh\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency W-SUN will be used.

c:\Users\vaibh\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency W-SUN will be used.

c:\Users\vaibh\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency W-SUN will be used.

c:\Users\vaibh\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequency information was provided, so inferred frequency W-SUN will be used.

c:\Users\vaibh\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning:

No frequ

In [13]:
import warnings
from statsmodels.tools.sm_exceptions import ValueWarning

# Suppress specific ValueWarnings from statsmodels
warnings.simplefilter('ignore', ValueWarning)

def plot_forecast(df, category):
    top_2 = df.sum().nlargest(2).index  # Select the top 2 brands based on total search interest
    fig = go.Figure()

    for product in top_2:
        # Forecasting with ARIMA for the next 4 months
        model = ARIMA(df[product], order=(5, 1, 0))  # You can tweak the (p, d, q) values as needed
        model_fit = model.fit()

        # Forecast for the next 4 months
        forecast = model_fit.forecast(steps=4)
        
        # Create a new date index for the forecasted values (4 months)
        forecast_index = pd.date_range(start=df.index[-1] + pd.Timedelta(days=1), periods=4, freq='M')
        
        # Plot only the forecast data (exclude past data)
        fig.add_trace(go.Scatter(x=forecast_index, y=forecast, mode='lines', name=f'Forecast {product}', line=dict(dash='dash')))
    
    # Customize the layout of the plot
    fig.update_layout(
        title=f'4-Month Forecast for Top 2 Searched {category} Items ({state_name})',
        xaxis_title='Date',
        yaxis_title='Search Interest',
        legend_title=f'{category} Item',
        xaxis_tickangle=-45
    )
    
    fig.show()


In [14]:
# Plot forecast for men's wear
plot_forecast(mens_wear_data, 'Men\'s Wear')

# Plot forecast for women's wear
plot_forecast(womens_wear_data, 'Women\'s Wear')
